### Scryfall MKM Price Trend Harvester

This program extracts mtg card price data (MKM Trend) at the time the code is run.

Make sure a folder named 'cache' and a csv file names 'portfolio.csv' exist in this file's host folder before executing. Note, that the portfolio.csv file format can be seen in the repo where the current file is located on GitHub at https://github.com/craiggoldie/magic-the-gathering-scryfall-api-data2

In [30]:
import requests
import pandas as pd
import datetime
import time
import os
import re

# #Change Jupyter cell width to 100% of browser (uncomment if using small screen!)
# from IPython.core.display import display, HTML
# display(HTML("<style> .container { width:100% !important; }</style>"))
# display(HTML('<style> div.prompt {display:none} </style>'))

#Switch jupyter to enable multiple outputs per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [31]:
#store current date (for filename later)
currdate=datetime.datetime.today().strftime('%Y%m%d')

#### Use the API to get MKM card prices
See Scryfal REST API Docs at https://scryfall.com/docs/api

Initially I tried to go straight into the cards endpoint to grab all card data--but it is limited to 175 cards per page (not sure if there is a way of working through pages); so I decided to limit the request to one set at a time. In order to do that I need a list of all the set names, which I've done by making a request on the sets endpoint.

In [32]:
#Cards endpoint is throttled at 175 cards per go; so will get sets to cycle through
response=requests.get('https://api.scryfall.com/sets')
print(response.status_code) #should be 200=ok
results_sets=response.json() #translate the response object's content from bytes to dictionary object

200


The code below reveals what elements are available from the sets endpoint, for each set (code, name, type, card_count, etc.)

In [33]:
#investigate what fields are available
results_sets['data'][0] #comment out if convenient

{'object': 'set',
 'code': 'gnt',
 'name': 'Game Night',
 'uri': 'https://api.scryfall.com/sets/gnt',
 'scryfall_uri': 'https://scryfall.com/sets/gnt',
 'search_uri': 'https://api.scryfall.com/cards/search?order=set&q=e%3Agnt&unique=prints',
 'released_at': '2018-11-16',
 'set_type': 'box',
 'card_count': 11,
 'digital': False,
 'foil_only': False,
 'icon_svg_uri': 'https://img.scryfall.com/sets/gnt.svg?1537761600'}

Now I want to loop through all the sets, and for each set I want to grab some data fields and put them in a data frame called sets. I've also added a line to export to csv for convenience only--(the csv is not used again later).

In [34]:
sets=pd.DataFrame()
for i in range(0,len(results_sets['data'])):
    #some sets don't have release dates (tokens, promos, etc. so added this workaround to get it)
    try:
        release_date=results_sets['data'][i]['released_at']
    except:
        release_date='01/01/1900'

    #cache results to sets dataframe
    df=pd.DataFrame({'set_code':[results_sets['data'][i]['code']],
        'set_name':results_sets['data'][i]['name'],
        'release_date':release_date,
        'set_type':results_sets['data'][i]['set_type'],
        'card_count':results_sets['data'][i]['card_count'],
        'digital':results_sets['data'][i]['digital']})
    sets=sets.append(df,ignore_index=True)
    #print(str(i)+':'+results_sets['data'][i]['name'])

sets.to_csv('sets_'+currdate+'.csv',index=False)

After gathering set data, I now go back and loop through each set_code, and for each set_code I insert the code into a GET request on the cards endpoint. The 'q' in the GET request string denotes that a query follows. You can see I've said to get all cards where set=s (the set_code), lang=en (English), and eur>2 (i.e. CardMarket trend price in Euros is greater than 2). The reason I added the last parameter was just to help keep me under that 175 page limit--as this made sense since I am not too fussed about monitoring cards less than €2 anyway, since it costs about that much to ship a card to a seller).

The below loop executes for every set (at the time the code runs) then saves trend prices to a csv file in the 'cache' folder. Every time the code runs it will write a file in the cache folder named after the current date.

In [35]:
#Use set_codes from above loop to query cards data in chunks
start=time.time()

cards=pd.DataFrame()

for s in sets.set_code:
    try:
        response=requests.get('https://api.scryfall.com/cards/search?q=set='+s+'+lang=en+eur>2')
        results_cards=response.json()
        #print('results_cards["data"] contains '+str(len(results_cards['data']))+' elements')

        #loop through the results and hive the good stuff
        for c in range(0,len(results_cards['data'])):
            try:
                card_name=results_cards['data'][c]['name']
                set_name=results_cards['data'][c]['set_name']
                set_tckr=results_cards['data'][c]['set']
                modern=results_cards['data'][c]['legalities']['modern']
                usd=results_cards['data'][c]['usd']
                eur=results_cards['data'][c]['eur']
                df=pd.DataFrame({'card_name':[card_name],
                    'set':set_tckr,
                    'set_name':set_name,
                    'modern':modern,
                    'usd':usd,
                    'eur':eur})
                cards=cards.append(df,ignore_index=True)
            except:
                continue
        print(s+' cached '+str(len(results_cards['data']))+' cards')
    except:
        print(s+' FAILED')
        continue

#convert cards eur and usd columns to numeric
cards['eur']=cards.eur.astype('float')
cards['usd']=cards.usd.astype('float')

cards=cards[~cards.duplicated(subset=['set','card_name'],keep='first')] #remove dups
cards.to_csv('cache/cards_'+currdate+'.csv',index=False)
end=time.time()
print(str((end-start)/60)+' mins')

gnt FAILED
pgp1 FAILED
pgrn FAILED
tgrn FAILED
grn cached 20 cards
gk1 FAILED
tgk1 FAILED
prwk FAILED
med FAILED
oc18 FAILED
tc18 FAILED
c18 cached 13 cards
htr17 FAILED
ps18 FAILED
ana FAILED
m19 cached 22 cards
tm19 FAILED
pm19 cached 1 cards
pss3 FAILED
gs1 cached 3 cards
ss1 cached 2 cards
pbbd FAILED
tcm2 FAILED
tbbd FAILED
cm2 cached 25 cards
bbd cached 22 cards
dom cached 16 cards
pdom cached 1 cards
tdom FAILED
tddu FAILED
ddu cached 4 cards
a25 cached 34 cards
ta25 FAILED
plny FAILED
pnat FAILED
trix FAILED
prix cached 1 cards
rix cached 19 cards
j18 cached 4 cards
f18 FAILED
ust cached 2 cards
tima FAILED
e02 cached 6 cards
v17 cached 10 cards
pxtc FAILED
ima cached 34 cards
tust FAILED
pust FAILED
ddt cached 5 cards
tddt FAILED
pgp17 FAILED
xln cached 20 cards
txln FAILED
pxln FAILED
pss2 FAILED
h17 cached 3 cards
htr FAILED
te01 FAILED
oc17 FAILED
c17 cached 20 cards
tc17 FAILED
ps17 FAILED
hou cached 9 cards
thou FAILED
phou cached 1 cards
e01 cached 11 cards
cma cached 18

The above log shows that a number of sets failed when executing the above code. I didn't do loads of research into this; but part of the reason was absence of a release date. I considered that if a set doesn't have a release date on scryfall it is either online, or otherwise of no interest to me. I saw the sets I know about coming through OK so just left the try/catch (above) in as-is.

There are a few other lines to clean up the dataframe; and then I cached all the data for this date in a csv file in the cache folder.

## Analytics

1. After gathering the above data for today, I can do some analysis to identify any valid **arbitrage** opportunities for buying on cardmarket and selling to a buylist in the US

2. After gathering over several iterations--for various different days over time--I can collate all previous snapshots into one view (using card name and set as the key)--then I can see price data per card as a **time series**. I can then use this to see up/down movements to predict future movements and recommend 'buy' decisions

3. I can also use the data to monitor performance of previously picked cards (my **portfolio**) and flag sell-triggers based on desired holding periods, etc.

#### Arbitrage Opportunities
Check to see if there is any scope to buy in eur and sell to US buylist

In [36]:
#Analyse 'today' Arbitrage Opportunities
xrate=0.86 #set xrate (usd-->eur)
pp=7 #set postage & packaging cost to me (to ship to us card kingdom)

arbitrage=cards.copy()
arbitrage['sell_gain']=(arbitrage.usd/2-pp)*xrate #buylist revenue to me in eur, assuming buylist=half us mkt cost
arbitrage['pp2me']=pp
arbitrage.loc[arbitrage.eur<10,'pp2me']=2
arbitrage['buy_cost']=arbitrage.eur+arbitrage.pp2me #cost of acquiring to me
arbitrage['margin_eu_us']=arbitrage.sell_gain-arbitrage.buy_cost

arbitrage.sort_values(by='margin_eu_us',ascending=False, inplace=True)
arbitrage.loc[(arbitrage.usd<200) & (arbitrage.modern=='legal')].head(5)

,card_name,set,set_name,modern,usd,eur,sell_gain,pp2me,buy_cost,margin_eu_us
3272,Circle of Protection: Green,lea,Limited Edition Alpha,legal,49.99,7.00,15.4757,2,9.00,6.4757
2990,Mahamoti Djinn,2ed,Unlimited Edition,legal,58.16,13.00,18.9888,7,20.00,-1.0112
1910,Nature's Will,chk,Champions of Kamigawa,legal,17.99,3.55,1.7157,2,5.55,-3.8343
1662,Wound Reflection,shm,Shadowmoor,legal,21.99,5.63,3.4357,2,7.63,-4.1943
2210,Relentless Assault,ptk,Portal Three Kingdoms,legal,49.99,13.34,15.4757,7,20.34,-4.8643


Apparently if I buy an Alpha Edition 'Circle of Protection: Green' on cardmarket, I can buylist it to a US vendor (e.g. CardKingdom), and make a profit of €6.47 (assuming buylist prices are half usd and shipping to US is $7 and shipping from Europe is €2. I would then go on cardmarket and check CardKingdom to see if this is legitimate.

#### Trend Opportunities
Look at price trend movements to identify buy opportunities

In [37]:
#collate cache data into one dataframe (eur prices only)
cache_df=pd.DataFrame(columns={'card_name','set','set_name'})
for i in os.listdir('cache'):
    df=pd.read_csv('cache/'+i,
                   usecols=['card_name','set','set_name','eur'],
                   dtype={'eur':'float'})
    datestr=re.findall(r'\d+',i) #pull number chunks from string
    df.columns.values[3]='eur_'+datestr[0]
    cache_df=cache_df.merge(df,on=['card_name','set','set_name'],how='outer')
    print(i+'-->cache_df.shape='+str(cache_df.shape))

#remove any duplicates (should not be necessary)
cache_df=cache_df[~cache_df.duplicated(subset=['set','card_name'],keep='first')]
print('deduped-->cache_df.shape='+str(cache_df.shape))

cards_20180917.csv-->cache_df.shape=(3415, 4)
cards_20180918.csv-->cache_df.shape=(3440, 5)
cards_20180919.csv-->cache_df.shape=(3463, 6)
cards_20180921.csv-->cache_df.shape=(3490, 7)
cards_20180924.csv-->cache_df.shape=(3534, 8)
cards_20180925.csv-->cache_df.shape=(3539, 9)
cards_20180928.csv-->cache_df.shape=(3555, 10)
deduped-->cache_df.shape=(3524, 10)


In [38]:
#see gain % and value between two dates
def add_gscore(date0, date1, nametag):
    df=cache_df.copy()
    df['gain_pct']=(df['eur_'+date1]/df['eur_'+date0])-1
    df['gain_val']=(df['eur_'+date1]-df['eur_'+date0])
    df['gain_score_'+nametag]=df.gain_val*abs(df.gain_pct) #add gain_score variable
    df['gain_val_'+nametag]=df.gain_val #add gain_val variable
    return cache_df.merge(df[['card_name','set','set_name','gain_val_'+nametag]],on=['card_name','set','set_name']),\
            date0

In [39]:
#apply the above function for some discrete periods
cache_df, date0=add_gscore('20180917','20180928','1week')
cache_df, date0=add_gscore('20180917','20180919','2days')

In [40]:
#add column with no. of distinct trend vals variable
cache_df['unqcols']=cache_df.nunique(axis=1) #this is the number of distinct trend price values--a proxy for traded volume

To view the output I've applied some rules to home in on valid opportunities:
* only looking for data with no null values
* not keeping really high value cards (I'm not willing to part with that much on a card--too risky)
* only keeping if the trend price is moving each day--indicating more frequent trades for that card

I'm then sorting by trading value gained in descending order and just looking at the top five records returned.

In [41]:
#view outcome (removing any cases where each day does not have a distinct value)
ncols=len(os.listdir('cache'))+5

#drop rows with any nulls
#only keep if original value<50
#only keep if trend price changes daily
#sort by 1wk score
#only keep top 5
cache_df.\
    dropna().\
    loc[(cache_df['eur_'+date0]<50) & \
        (cache_df.unqcols==ncols)].\
    sort_values(by=cache_df.columns[-3],ascending=False).\
    head(5)

,card_name,set,set_name,eur_20180917,eur_20180918,eur_20180919,eur_20180921,eur_20180924,eur_20180925,eur_20180928,gain_val_1week,gain_val_2days,unqcols
2289,Back to Basics,usg,Urza's Saga,37.58,38.18,38.41,38.79,45.60,45.28,46.21,8.63,0.83,12
438,Sakashima's Student,pca,Planechase Anthology,10.55,10.70,10.77,10.92,13.22,13.24,13.35,2.80,0.22,12
1742,Horizon Canopy,fut,Future Sight,49.41,49.49,49.76,49.87,50.38,50.54,52.07,2.66,0.35,12
3195,Island,leb,Limited Edition Beta,25.12,25.72,26.09,25.85,26.11,27.11,27.42,2.30,0.97,12
412,Verdant Catacombs,mm3,Modern Masters 2017,36.26,36.86,35.95,36.72,37.59,38.06,38.26,2.00,-0.31,12


Apparently, 'Back to Basics' from Urza's Saga has gained €8.63 this week and would be a good candidate to look into. I could either buy it and hold for a month, then see if it turns a profit; or I can add it to my training/simulated portfolio and monitor how the price moves going forward--to get more market insight before making these kind of buy decisions.

That takes me to portfolio monitoring.

#### Portfolio Monitoring
Monitor prices of all cards in portfolio (simulated or owned) to determine sell triggers (where cleared +€3 since buy date)

To do this, I have used my portfolio.csv list of cards owned then left-joined to today's cards dataframe, then done some calculations to show value movement since the buydate.

In [42]:
#read in portfolio dataset
pfolio=pd.read_csv('portfolio.csv', parse_dates=['buydate'], dayfirst=True, dtype={'paid':'float'})

#merge pfolio to current card trend prices
pfolio=pfolio.merge(cards[['card_name','set_name','eur']], on=['card_name','set_name'], how='left')

#movement since buy date
pfolio['movement']=pfolio.eur-pfolio.paid
pfolio['held_days']=(datetime.datetime.now()-pfolio.buydate).dt.days

#view results
pfolio.sort_values(by='movement',ascending=False).head(10)

,buydate,card_name,set_name,paid,qty,eur,movement,held_days
1,2018-07-01,Wooded Foothills,Khans of Tarkir,11.0,5,16.05,5.05,89
5,2018-07-01,Tireless Tracker,Shadows over Innistrad,4.0,2,9.05,5.05,89
6,2018-07-01,"Ghalta, Primal Hunger",Rivals of Ixalan,2.5,1,6.58,4.08,89
0,2018-07-01,Rekindling Phoenix,Rivals of Ixalan,16.0,2,18.05,2.05,89
20,2018-07-01,Tendershoot Dryad,Rivals of Ixalan,2.0,1,3.93,1.93,89
31,2018-07-01,Sorcerous Spyglass,Ixalan,0.9,1,2.70,1.80,89
19,2018-07-01,Tolaria,Legends,2.0,1,3.65,1.65,89
17,2018-07-01,"Nissa, Vital Force",Kaladesh,1.5,2,2.72,1.22,89
32,2018-07-01,Field of Ruin,Ixalan,2.0,1,3.07,1.07,89
8,2018-07-01,Sol Ring,Revised Edition,4.0,1,4.98,0.98,89


According to the above data, the cards with the most movement since buying are shown. I would make €5.05 if on my what I paid for copies of Wooded Foothills if I sold them today.